In [1]:
from pyspark.sql import SparkSession

In [2]:
# Caminho para o jar do sqlite
caminho_jar_sqlite = "../drivers/sqlite-jdbc-3.44.1.0.jar"

# Criando uma sessão spark
spark = SparkSession.builder.appName("modelling").config("spark.jars", caminho_jar_sqlite).getOrCreate()

23/11/29 11:15:46 WARN Utils: Your hostname, daniel-VJFE43F11X-XXXXXX resolves to a loopback address: 127.0.1.1; using 192.168.0.157 instead (on interface wlo1)
23/11/29 11:15:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/11/29 11:15:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/29 11:15:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Lendo o arquivo csv
dados = spark.read.csv("../data/raw/chamada_regular_sisu_2022_1.csv", header=True, sep="|", encoding="latin1")

In [4]:
# Observando o estado inicial dos dados
dados.show(5)

23/11/29 11:16:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------+-----+---------------+----------+--------------------+---------+------+-------------+--------------------+---------+----------------+------------+--------------------+-----------+-------+----------------+-------+---------------------+------------------+---------------------+----------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+-------------+--------------+--------------------+----+---------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+--------------+
| ANO|EDICAO|ETAPA|       DS_ETAPA|CODIGO_IES|            NOME_IES|SIGLA_IES|UF_IES|CODIGO_CAMPUS|         NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|CODIGO_CURSO|          NOME_CURSO|       GRAU|  TURNO|DS_PERIODICIDADE|TP_COTA|TIPO_MOD_CONCORRENCIA|  MOD_CONCORRENCIA|QT_VAGAS_CONC

In [5]:
# Checando o schema dos dados
dados.printSchema()

root
 |-- ANO: string (nullable = true)
 |-- EDICAO: string (nullable = true)
 |-- ETAPA: string (nullable = true)
 |-- DS_ETAPA: string (nullable = true)
 |-- CODIGO_IES: string (nullable = true)
 |-- NOME_IES: string (nullable = true)
 |-- SIGLA_IES: string (nullable = true)
 |-- UF_IES: string (nullable = true)
 |-- CODIGO_CAMPUS: string (nullable = true)
 |-- NOME_CAMPUS: string (nullable = true)
 |-- UF_CAMPUS: string (nullable = true)
 |-- MUNICIPIO_CAMPUS: string (nullable = true)
 |-- CODIGO_CURSO: string (nullable = true)
 |-- NOME_CURSO: string (nullable = true)
 |-- GRAU: string (nullable = true)
 |-- TURNO: string (nullable = true)
 |-- DS_PERIODICIDADE: string (nullable = true)
 |-- TP_COTA: string (nullable = true)
 |-- TIPO_MOD_CONCORRENCIA: string (nullable = true)
 |-- MOD_CONCORRENCIA: string (nullable = true)
 |-- QT_VAGAS_CONCORRENCIA: string (nullable = true)
 |-- PERCENTUAL_BONUS: string (nullable = true)
 |-- PESO_L: string (nullable = true)
 |-- PESO_CH: string 

In [6]:
# Permitindo usar SQL nos dados
dados.createOrReplaceTempView("dados")

Os dados aparentam estar em um bom estados, onde as únicas alterações exigidas 
são a mudança do tipo de dado de algumas colunas e a substituição do "." por ","
em todos os valores de notas, além da mudança de tipo dos mesmos.

In [7]:
# Aplicando as correções
dados_transformados = spark.sql("SELECT CAST(ANO AS INT) AS ANO, \
                                CAST(EDICAO AS INT) AS EDICAO, \
                                CAST(ETAPA AS INT) AS ETAPA, \
                                DS_ETAPA, \
                                CAST(CODIGO_IES AS INT) AS CODIGO_IES, \
                                NOME_IES, \
                                SIGLA_IES, \
                                UF_IES, \
                                CAST(CODIGO_CAMPUS AS INT) AS CODIGO_CAMPUS, \
                                NOME_CAMPUS, \
                                UF_CAMPUS, \
                                MUNICIPIO_CAMPUS, \
                                CAST(CODIGO_CURSO AS INT) AS CODIGO_CURSO, \
                                NOME_CURSO, \
                                GRAU, \
                                TURNO, \
                                DS_PERIODICIDADE, \
                                CASE WHEN TP_COTA IS NULL THEN 'Não se aplica' ELSE TP_COTA END AS TP_COTA, \
                                TIPO_MOD_CONCORRENCIA, \
                                MOD_CONCORRENCIA, \
                                CAST(QT_VAGAS_CONCORRENCIA AS INT) AS QT_VAGAS_CONCORRENCIA, \
                                CAST(PERCENTUAL_BONUS AS INT) AS PERCENTUAL_BONUS, \
                                PESO_L, \
                                PESO_CH, \
                                PESO_CN, \
                                PESO_M, \
                                PESO_R, \
                                CAST(REPLACE(NOTA_MINIMA_L, ',', '.') AS FLOAT) AS NOTA_MINIMA_L, \
                                CAST(REPLACE(NOTA_MINIMA_CH, ',', '.') AS FLOAT) AS NOTA_MINIMA_CH, \
                                CAST(REPLACE(NOTA_MINIMA_CN, ',', '.') AS FLOAT) AS NOTA_MINIMA_CN, \
                                CAST(REPLACE(NOTA_MINIMA_M, ',', '.') AS FLOAT) AS NOTA_MINIMA_M, \
                                CAST(REPLACE(NOTA_MINIMA_R, ',', '.') AS FLOAT) AS NOTA_MINIMA_R, \
                                CAST(REPLACE(MEDIA_MINIMA, ',', '.') AS FLOAT) AS MEDIA_MINIMA, \
                                CPF, \
                                INSCRICAO_ENEM, \
                                INSCRITO, \
                                SEXO, \
                                CAST(TO_DATE(DATA_NASCIMENTO, 'dd/MM/yyyy') AS DATE) AS DATA_NASCIMENTO, \
                                UF_CANDIDATO, \
                                MUNICIPIO_CANDIDATO, \
                                OPCAO, \
                                CAST(REPLACE(NOTA_L, ',', '.') AS FLOAT) AS NOTA_L, \
                                CAST(REPLACE(NOTA_CH, ',', '.') AS FLOAT) AS NOTA_CH, \
                                CAST(REPLACE(NOTA_CN, ',', '.') AS FLOAT) AS NOTA_CN, \
                                CAST(REPLACE(NOTA_M, ',', '.') AS FLOAT) AS NOTA_M, \
                                CAST(REPLACE(NOTA_R, ',', '.') AS FLOAT) AS NOTA_R, \
                                CAST(REPLACE(NOTA_L_COM_PESO, ',', '.') AS FLOAT) AS NOTA_L_COM_PESO, \
                                CAST(REPLACE(NOTA_CH_COM_PESO, ',', '.') AS FLOAT) AS NOTA_CH_COM_PESO, \
                                CAST(REPLACE(NOTA_CN_COM_PESO, ',', '.') AS FLOAT) AS NOTA_CN_COM_PESO, \
                                CAST(REPLACE(NOTA_M_COM_PESO, ',', '.') AS FLOAT) AS NOTA_M_COM_PESO, \
                                CAST(REPLACE(NOTA_R_COM_PESO, ',', '.') AS FLOAT) AS NOTA_R_COM_PESO, \
                                CAST(REPLACE(NOTA_CANDIDATO, ',', '.') AS FLOAT) AS NOTA_CANDIDATO, \
                                CAST(REPLACE(NOTA_CORTE, ',', '.') AS FLOAT) AS NOTA_CORTE, \
                                CLASSIFICACAO, \
                                APROVADO, \
                                MATRICULA \
                                FROM dados")

In [8]:
# Salvando os dados transformados n formato de database
dados_transformados.write.format("jdbc") \
    .option("url", "jdbc:sqlite:{}".format("../data/processed/sisu.db")) \
    .option("dbtable", "data_sisu") \
    .mode("overwrite") \
    .save()

Py4JJavaError: An error occurred while calling o43.save.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:109)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:109)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:47)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.c